In [10]:
import pandas as pd
import numpy as np
import cv2
import os

In [11]:
filelist_df = pd.read_csv('EchoNet-Dynamic/FileList.csv')

# Cargar el archivo VolumeTracings.csv
volumetracings_df = pd.read_csv('EchoNet-Dynamic/VolumeTracings.csv')

In [13]:
videos = ['0X100009310A3BD7FC.avi', '0X1002E8FBACD08477.avi', '0X1005D03EED19C65B.avi', '0X10075961BC11C88E.avi', '0X10094BA0A028EAC3.avi', 
          '0X100CF05D141FF143.avi', '0X100E3B8D3280BEC5.avi', '0X100E491B3CD58DE2.avi', '0X100F044876B98F90.avi', '0X101026B90DAE7E95.avi']
frames = [46, 3, 24, 91, 137, 132, 25, 49, 56, 45]

### Rectas

In [17]:
import numpy as np
import cv2
import os

# Inicializa una lista para almacenar los puntos
all_points = []

for i in range(len(videos)):

    # Crea el video_path
    video_path = os.path.join("EchoNet-Dynamic/Videos", videos[i])
    file_name = os.path.basename(video_path)
    
    # Obtiene el frame de interés
    frame_of_interest = frames[i]

    # Abre el video
    cap = cv2.VideoCapture(video_path)

    # Verifica si el video se abrió correctamente
    if not cap.isOpened():
        print(f"Error al abrir el video {video_path}")
    else:
        frame_count = 0

        while True:
            ret, frame = cap.read()

            if not ret:
                break  # Fin del video

            if frame_count == frame_of_interest:
                # Define la carpeta donde se guardará la imagen
                folder_path = "images"

                # Verifica si la carpeta existe, si no, la crea
                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)

                # Guarda el frame como una imagen en la carpeta "images"
                frame_filename = os.path.join(folder_path, f"frame_{frame_of_interest}_{i}.jpg")
                cv2.imwrite(frame_filename, frame)

                print(f"Frame {frame_of_interest} del video {videos[i]} guardado como {frame_filename}")

                # Recolecta los puntos en lugar de crear la máscara inmediatamente
                mask_coords = volumetracings_df[volumetracings_df['FileName'] == file_name]
                
                current_points = []  # Almacenar los puntos actuales
                for index, row in mask_coords.iterrows():
                    x1, y1, x2, y2 = row['X1'], row['Y1'], row['X2'], row['Y2']
                    current_points.extend([(x1, y1), (x2, y2)])

                color=(0, 0, 0)
                all_points.append((current_points, color))  # Agrega los puntos actuales y el color a la lista

                break

            frame_count += 1

        # Libera la captura
        cap.release()

# Después de recolectar todos los puntos, conectar cada punto con su vértice más cercano
for i, (points, color) in enumerate(all_points):
    # Crear una máscara en blanco y negro
    mask = np.zeros_like(frame, dtype=np.uint8)

    # Conectar cada punto con su vértice más cercano
    for j in range(0, len(points), 2):
        # Determina el color en función de la posición de la iteración
        if j == 0:
            color = (0, 255, 0)  # Verde para la primera iteración
        elif 1 <= j <= 40:
            color = (0, 0, 255)  # Azul para las iteraciones del 2 al 11
        elif j == 42:
            color = (0, 255, 0)
        else:
            color = (255, 0, 0)  # Rojo para las iteraciones restantes

        x1, y1 = map(int, points[j])  # Convertir las coordenadas a enteros
        x2, y2 = map(int, points[j + 1])  # Convertir las coordenadas a enteros
        cv2.line(mask, (x1, y1), (x2, y2), color, 1)  # Conectar puntos con el color correspondiente

    # Guarda la máscara
    mask_filename = os.path.join("images", f"frame_{frames[i]}_lineMask.jpg")
    cv2.imwrite(mask_filename, mask)

    print(f"Máscara del frame {frames[i]} del video {videos[i]} guardada como {mask_filename}")


Frame 46 del video 0X100009310A3BD7FC.avi guardado como images/frame_46_0.jpg
Frame 3 del video 0X1002E8FBACD08477.avi guardado como images/frame_3_1.jpg
Frame 24 del video 0X1005D03EED19C65B.avi guardado como images/frame_24_2.jpg
Frame 91 del video 0X10075961BC11C88E.avi guardado como images/frame_91_3.jpg
Frame 137 del video 0X10094BA0A028EAC3.avi guardado como images/frame_137_4.jpg
Frame 132 del video 0X100CF05D141FF143.avi guardado como images/frame_132_5.jpg
Frame 25 del video 0X100E3B8D3280BEC5.avi guardado como images/frame_25_6.jpg
Frame 49 del video 0X100E491B3CD58DE2.avi guardado como images/frame_49_7.jpg
Frame 56 del video 0X100F044876B98F90.avi guardado como images/frame_56_8.jpg
Frame 45 del video 0X101026B90DAE7E95.avi guardado como images/frame_45_9.jpg
Máscara del frame 46 del video 0X100009310A3BD7FC.avi guardada como images/frame_46_lineMask.jpg
Máscara del frame 3 del video 0X1002E8FBACD08477.avi guardada como images/frame_3_lineMask.jpg
Máscara del frame 24 del v

### Polígono

In [6]:
import numpy as np
import cv2
import os

# Inicializa una lista para almacenar los puntos
all_points = []

for i in range(len(videos)):
    # Crea el video_path
    video_path = os.path.join("EchoNet-Dynamic/Videos", videos[i])
    file_name = os.path.basename(video_path)
    
    # Obtiene el frame de interés
    frame_of_interest = frames[i]

    # Abre el video
    cap = cv2.VideoCapture(video_path)

    # Verifica si el video se abrió correctamente
    if not cap.isOpened():
        print(f"Error al abrir el video {video_path}")
    else:
        frame_count = 0

        while True:
            ret, frame = cap.read()

            if not ret:
                break  # Fin del video

            if frame_count == frame_of_interest:
                # Define la carpeta donde se guardará la imagen
                folder_path = "images"

                # Verifica si la carpeta existe, si no, la crea
                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)

                # Guarda el frame como una imagen en la carpeta "images"
                frame_filename = os.path.join(folder_path, f"frame_{frame_of_interest}_{i}.jpg")
                cv2.imwrite(frame_filename, frame)

                print(f"Frame {frame_of_interest} del video {videos[i]} guardado como {frame_filename}")

                # Recolecta los puntos en lugar de crear la máscara inmediatamente
                mask_coords = volumetracings_df[volumetracings_df['FileName'] == file_name]
                
                current_points = []  # Almacenar los puntos actuales
                for index, row in mask_coords.iterrows():
                    x1, y1, x2, y2 = row['X1'], row['Y1'], row['X2'], row['Y2']
                    current_points.extend([(x1, y1), (x2, y2)])

                all_points.append(current_points)  # Agrega los puntos actuales a la lista

                break

            frame_count += 1

        # Libera la captura
        cap.release()

# Después de recolectar todos los puntos, conectar cada punto con su vértice más cercano
for i, points in enumerate(all_points):
    # Crear una máscara en blanco y negro
    mask = np.zeros_like(frame, dtype=np.uint8)

    # Convertir los puntos en un formato adecuado para cv2.polylines
    polygon_points = np.array(points, dtype=np.int32).reshape((-1, 1, 2))

    # Dibujar el polígono en la máscara
    cv2.polylines(mask, [polygon_points], isClosed=True, color=(255, 255, 255), thickness=3)

    # Guardar la máscara
    mask_filename = os.path.join("images", f"frame_{frames[i]}_polyMask.jpg")
    cv2.imwrite(mask_filename, mask)

    print(f"Máscara del frame {frames[i]} del video {videos[i]} guardada como {mask_filename}")

Frame 46 del video 0X100009310A3BD7FC.avi guardado como images/frame_46_0.jpg
Frame 3 del video 0X1002E8FBACD08477.avi guardado como images/frame_3_1.jpg
Frame 24 del video 0X1005D03EED19C65B.avi guardado como images/frame_24_2.jpg
Frame 91 del video 0X10075961BC11C88E.avi guardado como images/frame_91_3.jpg
Frame 137 del video 0X10094BA0A028EAC3.avi guardado como images/frame_137_4.jpg
Frame 132 del video 0X100CF05D141FF143.avi guardado como images/frame_132_5.jpg
Frame 25 del video 0X100E3B8D3280BEC5.avi guardado como images/frame_25_6.jpg
Frame 49 del video 0X100E491B3CD58DE2.avi guardado como images/frame_49_7.jpg
Frame 56 del video 0X100F044876B98F90.avi guardado como images/frame_56_8.jpg
Frame 45 del video 0X101026B90DAE7E95.avi guardado como images/frame_45_9.jpg
Máscara del frame 46 del video 0X100009310A3BD7FC.avi guardada como images/frame_46_polyMask.jpg
Máscara del frame 3 del video 0X1002E8FBACD08477.avi guardada como images/frame_3_polyMask.jpg
Máscara del frame 24 del v

### Poligonos divididos (EDV / ESV)

In [18]:
import numpy as np
import cv2
import os

# Inicializa una lista para almacenar los puntos
all_points = []

for i in range(len(videos)):

    # Crea el video_path
    video_path = os.path.join("EchoNet-Dynamic/Videos", videos[i])
    file_name = os.path.basename(video_path)
    
    # Obtiene el frame de interés
    frame_of_interest = frames[i]

    # Abre el video
    cap = cv2.VideoCapture(video_path)

    # Verifica si el video se abrió correctamente
    if not cap.isOpened():
        print(f"Error al abrir el video {video_path}")
    else:
        frame_count = 0

        while True:
            ret, frame = cap.read()

            if not ret:
                break  # Fin del video

            if frame_count == frame_of_interest:
                # Define la carpeta donde se guardará la imagen
                folder_path = "images"

                # Verifica si la carpeta existe, si no, la crea
                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)

                # Guarda el frame como una imagen en la carpeta "images"
                frame_filename = os.path.join(folder_path, f"frame_{frame_of_interest}_{i}.jpg")
                cv2.imwrite(frame_filename, frame)

                print(f"Frame {frame_of_interest} del video {videos[i]} guardado como {frame_filename}")

                # Recolecta los puntos en lugar de crear la máscara inmediatamente
                mask_coords = volumetracings_df[volumetracings_df['FileName'] == file_name]
                
                current_points = []  # Almacenar los puntos actuales
                for index, row in mask_coords.iterrows():
                    x1, y1, x2, y2 = row['X1'], row['Y1'], row['X2'], row['Y2']
                    current_points.extend([(x1, y1), (x2, y2)])

                all_points.append((current_points, color))  # Agrega los puntos actuales y el color a la lista

                break

            frame_count += 1

        # Libera la captura
        cap.release()

# Inicializa los colores
blue_color = (0, 0, 255)  # Azul
red_color = (255, 0, 0)  # Rojo

# Después de recolectar todos los puntos, crear dos imágenes diferentes con dos máscaras distintas
for i, (points, color) in enumerate(all_points):
    # Crear una máscara en blanco y negro
    mask_x1y1 = np.zeros_like(frame, dtype=np.uint8)
    mask_x2y2 = np.zeros_like(frame, dtype=np.uint8)

    # Conectar cada punto con su vértice más cercano
    for j in range(0, len(points), 2):
        x1, y1 = map(int, points[j])  # Convertir las coordenadas a enteros
        x2, y2 = map(int, points[j + 1])  # Convertir las coordenadas a enteros
        if j < 21 * 2:
            cv2.polylines(mask_x1y1, [np.array([(x1, y1), (x2, y2)], dtype=np.int32)], isClosed=True, color=blue_color, thickness=3)
        else:
            cv2.polylines(mask_x2y2, [np.array([(x1, y1), (x2, y2)], dtype=np.int32)], isClosed=True, color=red_color, thickness=3)

    # Guardar las máscaras en imágenes separadas
    mask_x1y1_filename = os.path.join("images", f"frame_{frames[i]}_polyMask_EDV.jpg")
    mask_x2y2_filename = os.path.join("images", f"frame_{frames[i]}_polyMask_ESV.jpg")

    cv2.imwrite(mask_x1y1_filename, mask_x1y1)
    cv2.imwrite(mask_x2y2_filename, mask_x2y2)

    print(f"Máscara x1y1 del frame {frames[i]} del video {videos[i]} guardada como {mask_x1y1_filename}")
    print(f"Máscara x2y2 del frame {frames[i]} del video {videos[i]} guardada como {mask_x2y2_filename}")


Frame 46 del video 0X100009310A3BD7FC.avi guardado como images/frame_46_0.jpg
Frame 3 del video 0X1002E8FBACD08477.avi guardado como images/frame_3_1.jpg
Frame 24 del video 0X1005D03EED19C65B.avi guardado como images/frame_24_2.jpg
Frame 91 del video 0X10075961BC11C88E.avi guardado como images/frame_91_3.jpg
Frame 137 del video 0X10094BA0A028EAC3.avi guardado como images/frame_137_4.jpg
Frame 132 del video 0X100CF05D141FF143.avi guardado como images/frame_132_5.jpg
Frame 25 del video 0X100E3B8D3280BEC5.avi guardado como images/frame_25_6.jpg
Frame 49 del video 0X100E491B3CD58DE2.avi guardado como images/frame_49_7.jpg
Frame 56 del video 0X100F044876B98F90.avi guardado como images/frame_56_8.jpg
Frame 45 del video 0X101026B90DAE7E95.avi guardado como images/frame_45_9.jpg
Máscara x1y1 del frame 46 del video 0X100009310A3BD7FC.avi guardada como images/frame_46_polyMask_EDV.jpg
Máscara x2y2 del frame 46 del video 0X100009310A3BD7FC.avi guardada como images/frame_46_polyMask_ESV.jpg
Máscar